In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import Sequential

In [2]:
img_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
################### TRAINING IMAGES #########################
face_train = img_generator.flow_from_directory('data/train_face',target_size=(512,512),shuffle=False,
                                               seed=40,save_format='jpg', batch_size=16, classes=None,
                                               class_mode=None)

comic_train = img_generator.flow_from_directory('data/train_cartoon',target_size=(512,512),shuffle=False,
                                                seed=40,save_format='jpg',batch_size=16, classes=None,
                                                class_mode=None)

In [ ]:
################## VALADITION IMAGES ########################
face_val = img_generator.flow_from_directory('data/val_face',target_size=(512,512),shuffle=False,
                                             seed=40,save_format='jpg',batch_size=16,classes=None,
                                             class_mode=None)

comic_val = img_generator.flow_from_directory('data/val_cartoon',target_size=(512,512),shuffle=False,
                                              seed=40,save_format='jpg',batch_size=16,classes=None,
                                              class_mode=None)

In [5]:
############### TO FEED THE DATA IN model.fit ############
train_generator=zip(face_train, comic_train)
val_generator=zip(face_val, comic_val)

In [ ]:
###################### AUTOENCODER MODEL ###############################
model=Sequential(name="real2comic")
model.add(keras.Input(shape=(512,512,3)))
model.add(layers.Conv2D(3,3,activation='relu', padding='same'))
model.add(layers.Conv2D(32,3,activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2), padding='same'))
model.add(layers.Conv2D(64,3,activation='relu', padding='same'))
model.add(layers.Conv2D(64,3,activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2), padding='same'))
model.add(layers.Conv2D(128,3,activation='relu', padding='same'))
model.add(layers.Conv2D(128,3,activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2,2), padding='same'))

model.add(layers.Conv2D(128,3,activation='relu', padding='same'))
model.add(layers.Conv2DTranspose(128,3,activation='relu', padding='same'))


model.add(layers.Conv2DTranspose(128,3,activation='relu',strides=2, padding='same'))
model.add(layers.Conv2DTranspose(64,3,activation='relu',strides=2, padding='same'))
model.add(layers.Conv2DTranspose(32,3,activation='relu',strides=2, padding='same'))
model.add(layers.Conv2DTranspose(3,3,activation='sigmoid', padding='same'))

model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])

model.summary()

In [ ]:
######################### TRAINING THE MODEL ########################
model.fit(x=train_generator, validation_data = val_generator,steps_per_epoch=530, validation_steps=94 ,epochs=25)

In [ ]:
model.save('real2comic.h5')